In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Load data from the text file
data = np.loadtxt('data/Debutanizer_Column_Data.txt', skiprows=1)

x_temp = data[:, :7]
y_temp = data[:, 7]

TRAIN_SIZE = 1200
VAL_SIZE = 400

x_new = np.zeros([2390, 10])
x_6 = x_temp[:, 4]
x_9 = (x_temp[:, 5] + x_temp[:, 6])/2
x_new[:, :5] = x_temp[4: 2394, :5]

x_new[:, 5] = x_6[3: 2393]
x_new[:, 6] = x_6[2: 2392]
x_new[:, 7] = x_6[1: 2391]
x_new[:, 8] = x_9[4: 2394]
x_new[:, 9] = y_temp[:2390]
#x_new[:, 9] = y_temp[3: 2393]
#x_new[:, 10] = y_temp[2: 2392]
#x_new[:, 11] = y_temp[1:2391]
#x_new[:, 12] = y_temp[:2390]
y_new = y_temp[4: 2394]
y_new = y_new.reshape([-1, 1])

train_x = x_new[:TRAIN_SIZE, :]
train_y = y_new[:TRAIN_SIZE]

val_x = x_new[TRAIN_SIZE:TRAIN_SIZE+VAL_SIZE, :]
val_y = y_new[TRAIN_SIZE:TRAIN_SIZE+VAL_SIZE]

test_x = x_new[TRAIN_SIZE+VAL_SIZE:2390, :]
test_y = y_new[TRAIN_SIZE+VAL_SIZE:2390]


In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

k_values = range(1, 10)
inertias = []  # Sum of squared distances of samples to their closest cluster center

for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(train_x)
    inertias.append(kmeans.inertia_)

# Plot the elbow curve
plt.plot(k_values, inertias, marker='o')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Sum of Squared Distances')
plt.title('Elbow Method')
plt.show()

In [ ]:
def clusters(clusters, X, Y):
    kmeans = KMeans(n_clusters=clusters, random_state=42)
    kmeans.fit(X)
    cluster_labels = kmeans.labels_
    centroids = kmeans.cluster_centers_
    clustered_input = []
    clustered_output = []
    for i in range(clusters):
        cluster_points = X[cluster_labels == i]
        cluster_output = Y[cluster_labels == i]
        clustered_input.append(cluster_points)
        clustered_output.append(cluster_output)
    return clustered_input, clustered_output, centroids



In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

def parameters(X, Y):
    
    W = []
    b = []
    for i in range(len(X)):
        cluster_data = X[i]
        cluster_output = Y[i]

        # Create and fit the linear regression model
        linear_regression = LinearRegression()
        linear_regression.fit(cluster_data, cluster_output)
        predictions = linear_regression.predict(cluster_data)
        mse = mean_squared_error(cluster_output, predictions)
        

    # Access the coefficients (parameters)
        coefficients = linear_regression.coef_
        intercept = linear_regression.intercept_
        W.append(coefficients)
        b.append(intercept)
    return W, b
    

In [ ]:
def fit_clusters(train_x, train_y, test_x,test_y, k):
    clustered_input, clustered_output, centroids = clusters(k, train_x, train_y )
    W, b = parameters(clustered_input, clustered_output)
    assigned_clusters = []
    y_predict = np.zeros(len(test_x))

    for i in range(len(test_x)):
        # Calculate distances to centroids
        distances = np.linalg.norm(test_x[i] - centroids, axis=1)
        # Assign data point to the cluster with the closest centroid
        closest_cluster = np.argmin(distances)
        assigned_clusters.append(closest_cluster)
        y_predict[i] = np.dot(W[closest_cluster], test_x[i].reshape(10, 1)) + b[closest_cluster]

    mse = np.mean(np.square(y_predict - test_y))
    
    return y_predict , mse

    

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Train the linear regression model on training data
linear_regression = LinearRegression()
linear_regression.fit(train_x, train_y)

# Predict the output for test data using the trained model
predictions = linear_regression.predict(test_x)

# Calculate the mean squared error (MSE) for the test predictions
mse = mean_squared_error(test_y, predictions)
print(linear_regression.coef_)
# Print the MSE for the test predictions
print("Mean Squared Error:", mse)